In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Create a new directory
os.makedirs('/content/drive/My Drive/main', exist_ok=True)

# Change to the new directory
os.chdir('/content/drive/My Drive/main')


Mounted at /content/drive


In [ ]:
!pip install ultralytics==8.0.196

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 30.1/107.7 GB disk)


In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

In [ ]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/YOLOV8/runs/detect/train/weights/best.pt conf=0.25 source='/content/drive/MyDrive/YOLOV8/Test1.mp4' save=True

Ultralytics YOLOv8.0.196 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 11127132 parameters, 0 gradients, 28.4 GFLOPs

video 1/1 (1/992) /content/drive/MyDrive/YOLOV8/Test1.mp4: 448x800 1 Doctor, 1 Patient, 677.4ms
video 1/1 (2/992) /content/drive/MyDrive/YOLOV8/Test1.mp4: 448x800 1 Doctor, 1 Patient, 603.1ms
video 1/1 (3/992) /content/drive/MyDrive/YOLOV8/Test1.mp4: 448x800 1 Doctor, 2 Patients, 564.8ms
video 1/1 (4/992) /content/drive/MyDrive/YOLOV8/Test1.mp4: 448x800 2 Patients, 559.0ms
video 1/1 (5/992) /content/drive/MyDrive/YOLOV8/Test1.mp4: 448x800 1 Doctor, 1 Patient, 555.1ms
video 1/1 (6/992) /content/drive/MyDrive/YOLOV8/Test1.mp4: 448x800 1 Doctor, 1 Patient, 573.6ms
video 1/1 (7/992) /content/drive/MyDrive/YOLOV8/Test1.mp4: 448x800 1 Doctor, 1 Nurse, 1 Patient, 572.8ms
video 1/1 (8/992) /content/drive/MyDrive/YOLOV8/Test1.mp4: 448x800 1 Doctor, 1 Nurse, 1 Patient, 551.7ms
video 1/1 (9/992) /content/drive/MyDrive/YOLOV8/Test1.m

In [ ]:
!pip install cvzone

In [ ]:
import cv2
import cvzone
import math
from ultralytics import YOLO
from IPython.display import display, Image
from PIL import Image as PILImage
import numpy as np

cap = cv2.VideoCapture('fall.mp4')

model = YOLO('yolov8s.pt')

classnames = []
with open('classes.txt', 'r') as f:
    classnames = f.read().splitlines()

frame_skip = 5  # Change this to process every nth frame
frame_count = 0

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % frame_skip != 0:
        continue

    frame = cv2.resize(frame, (980, 740))

    results = model(frame)

    for info in results:
        parameters = info.boxes
        for box in parameters:
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
            confidence = box.conf[0]
            class_detect = box.cls[0]
            class_detect = int(class_detect)
            class_detect = classnames[class_detect]
            conf = math.ceil(confidence * 100)

            # Implement fall detection using the coordinates x1, y1, x2, y2
            height = y2 - y1
            width = x2 - x1
            threshold = height - width

            if conf > 80 and class_detect == 'person':
                cvzone.cornerRect(frame, [x1, y1, width, height], l=30, rt=6)
                cvzone.putTextRect(frame, f'{class_detect}', [x1 + 8, y1 - 12], thickness=2, scale=2)

            if threshold < 0:
                cvzone.putTextRect(frame, 'Fall Detected', [x1 + 8, y1 - 50], thickness=2, scale=2)

    # Convert frame to display in Jupyter
    _, buffer = cv2.imencode('.jpg', frame)
    display(Image(data=buffer))

    if cv2.waitKey(1) & 0xFF == ord('t'):
        break

cap.release()
cv2.destroyAllWindows()
